In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

C:\Users\USER\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:
import numpy as np
import pandas as pd

In [2]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

In [4]:
model.build(input_shape = (None, 3))

In [5]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[-0.23271327, -0.01091275, -0.24526855, -0.28311324, -0.15847969,
         -0.09378399, -0.00290263,  0.19833124,  0.09009448,  0.12105903,
          0.10189354,  0.01844102,  0.1311363 ,  0.21590817, -0.24590749,
          0.0301913 ,  0.21820718,  0.04414338, -0.01455122,  0.16951442,
         -0.27732888, -0.24928635, -0.18442091, -0.10523011, -0.22602487,
         -0.11566068,  0.06144133,  0.0228779 ,  0.03345925, -0.240353  ,
          0.05440527, -0.05827929, -0.0975731 , -0.13812591, -0.28754675,
         -0.0211747 ,  0.13423556,  0.00120947,  0.2885905 , -0.25823504,
         -0.09130509,  0.2614901 , -0.01828334,  0.23945409,  0.28696305,
          0.07985055, -0.26289052, -0.09601095, -0.2697456 , -0.19373167,
         -0.01763994,  0.24599129, -0.1580367 ,  0.08169073,  0.13834101,
         -0.1683962 , -0.14904585,  0.1766068 ,  0.26309907, -0.10577099,
          0.10814217, -0.03690818,  0.2562701

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# 함수형

inputs = keras.Input(shape=(3,),name="my_input")
features = layers.Dense(64, activation = "relu")(inputs)
outputs = layers.Dense(10,activation="softmax")(features)
model = keras.Model(inputs = inputs, outputs=outputs)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_2 (Dense)             (None, 64)                256       
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
vocabulary_size=10000
num_tags=100
num_departments=4

In [14]:
title=keras.Input(shape=(vocabulary_size,),name="title")
text_body=keras.Input(shape=(vocabulary_size,),name="text_body")
tags=keras.Input(shape=(num_tags,),name='tags')

features = layers.Concatenate()([title,text_body,tags])
features=layers.Dense(64, activation="relu")(features)

priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department=layers.Dense(
    num_departments, activation="softmax", name="department")(features)
    
model = keras.Model(inputs=[title, text_body, tags],
                    outputs=[priority, department])

In [16]:
num_samples=1280

In [18]:
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data=np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data=np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data=np.random.random(size=(num_samples, 1))
department_data=np.random.randint(0, 2, size=(num_samples, num_departments))

In [20]:
model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit([title_data, text_body_data, tags_data],
         [priority_data, department_data],
         epochs=1)

model.evaluate([title_data, text_body_data, tags_data],
               [priority_data, department_data])

priority_preds, department_preds = model.predict(
    [title_data, text_body_data, tags_data])

40/40 [==============================] - 0s 7ms/step


In [22]:
model.compile(optimizer="rmsprop",
              loss={"priority" : "mean_squared_error", "department" : "categorical_crossentropy"},
              metrics={"priority" : ["mean_absolute_error"], "department" : ["accuracy"]})

model.fit({"title" : title_data, "text_body" : text_body_data, "tags" : tags_data},
         {"priority" : priority_data, "department" : department_data},
         epochs=1)

model.evaluate({"title" : title_data, "text_body" : text_body_data, "tags" : tags_data},
               {"priority" : priority_data, "department" : department_data})

priority_preds, department_preds = model.predict(
    {"title" : title_data, "text_body" : text_body_data, "tags" : tags_data})

40/40 [==============================] - 0s 7ms/step


In [23]:
model.layers

In [24]:
model.layers[3].input

[<KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'title')>,
 <KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'text_body')>,
 <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'tags')>]

In [25]:
model.layers[3].output

<KerasTensor: shape=(None, 20100) dtype=float32 (created by layer 'concatenate_1')>

In [28]:
features = model.layers[4].output
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)
new_model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department, difficulty])

In [30]:
#!pip install pydot

DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\huggingface_hub-0.23.4-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [31]:
keras.utils.plot_model(
    new_model, "updated_ticket_classifier.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [33]:
class CustomerTicketModel(keras.Model):
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer= layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(num_departments, activation='softmax')
        
    def call(self, inputs):
        title = inputs["title"]
        text_body=inputs["text_body"]
        tags=inputs["tags"]
        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department

In [34]:
model = CustomerTicketModel(num_departments=4)
priority, department=model(
    {"title" : title_data, "text_body" : text_body_data, "tags" : tags_data})

In [36]:
model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit({"title" : title_data, "text_body" : text_body_data, "tags" : tags_data},
         [priority_data,department_data],
         epochs=1)

model.evaluate({"title" : title_data, "text_body" : text_body_data, "tags" : tags_data},
               [priority_data,department_data])

priority_preds, department_preds = model.predict(
    {"title" : title_data, "text_body" : text_body_data, "tags" : tags_data})

40/40 [==============================] - 0s 4ms/step


In [39]:
class Classifier(keras.Model):
    def __init__(self, num_classes=2):
        super().__init__()
        if num_classes == 2:
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"
        self.dense = layers.Dense(num_units, activation = activation)
    def call(self, inputs):
        return self.dense(inputs)

In [40]:
inputs = keras.Input(shape=(3,))
features = layers.Dense(64, activation = "relu")(inputs)
outputs = Classifier(num_classes = 10)(features)
model = keras.Model(inputs = inputs, outputs=outputs)

In [41]:
inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation = "sigmoid")(inputs)
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)

In [42]:
class MyModel(keras.Model):
    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation = "relu")
        self.classifier = binary_classifier
    def call(self, inputs):
        features = self.dense(inputs)
        return self.classifier(features)

In [43]:
model = MyModel()

In [ ]:
# 273p

In [45]:
from tensorflow.keras.datasets import mnist

In [46]:
def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation = "relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10,activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

In [49]:
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype('float32') /255
test_images = test_images.reshape((10000, 28*28)).astype('float32') /255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [51]:
model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer=keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

In [52]:
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss=loss_fn(targets, predictions)
    gradients=tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs["loss"] = loss_tracking_metric.result()
    return logs

In [53]:
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

In [54]:
training_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels))
training_dataset = training_dataset.batch(32)
epochs = 3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"{epoch}번째 에포크 결과")
    for key, value in logs.items():
        print(f"...{key} : {value:.4f}")

0번째 에포크 결과
...sparse_categorical_accuracy : 0.9143
...loss : 0.2905


KeyboardInterrupt: 

In [ ]:
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss=loss_fn(targets, predictions)
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
        
    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return.logs

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices(
    (val_images, val_labels))
training_dataset = training_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("평가 결과 : ")
for key, value in logs.items():
    print(f"...{key} : {value:.4f}")

In [ ]:
@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss=loss_fn(targets, predictions)
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
        
    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return.logs

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices(
    (val_images, val_labels))
training_dataset = training_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("평가 결과 : ")
for key, value in logs.items():
    print(f"...{key} : {value:.4f}")